In [6]:
import csv
import random
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec, Phrases
from keras.utils import pad_sequences
import nltk
import keras
nltk.download('stopwords')
nltk.download('wordnet')

In [2]:
# Lectura del dataset de train
trainfile = csv.reader(open("train.csv"), delimiter='\t')
trainrows = np.array([[c for c in row] for row in trainfile])
row_count_train, column_count = np.shape(trainrows)
T_train = [int(c) for c in trainrows[:, 0]]
P_train = trainrows[:, 1]

In [3]:
# Preprocesamiento de los textos
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = nltk.stem.WordNetLemmatizer()

# Eliminación de stop-wrods y stemming de los términos
P_train = [re.sub("[^a-zA-Z]", " ", l.lower()) for l in P_train]
P_train = [l.split() for l in P_train]
P_train = [[lemmatizer.lemmatize(l) for l in row if l not in stopwords] for row in P_train]

#Ensayo 1<BR>
Creación del modelo de word2vec.

In [ ]:
embedding_size = 256
model = Word2Vec(sentences = P_train, vector_size=embedding_size, min_count=3, window=5)

# Convertir features word2vec

vocab = model.wv.key_to_index
keys = list(vocab.keys())
filter_unknown = lambda word: vocab.get(word, None) is not None
encode = lambda review: list(map(keys.index, filter(filter_unknown, review)))
word_vector = list(map(encode, P_train))

input_length = 150
P = pad_sequences(sequences=word_vector, maxlen=input_length, padding='post')
T = np.array(T_train)
print(P.shape)
print(T.shape)

#Ensayo 2<BR>
Creación del modelo de bag of words

In [9]:
input_length = 1000
cv = CountVectorizer(max_features= input_length)
P_train_2 = ["".join(row) for row in P_train]
P_feat_train = cv.fit_transform(P_train_2).toarray()

cv._validate_vocabulary()

P_feat_train = ["".join(row) for row in P_train]
P = cv.transform(P_feat_train).toarray()
T = np.array(T_train)
print(P.shape)

(2758, 1000)


#Entrenamiento del modelo
Solo usar una preparación de los datos (Prep1 o Prep2)

In [ ]:
# Mezclar las muestras
inds = np.arange(len(T))
inds = np.random.permutation(inds)
P = P[inds]
T = T[inds]
print(P.shape)
print(T.shape)

In [ ]:
d_in = P.shape[1]
d_out = 1 # Clasificación binaria

modelo = keras.Sequential([
    keras.layers.Dense(5, input_shape=(d_in,), activation= 'tanh'),
    keras.layers.Dense(d_out, activation="sigmoid")
])

modelo.compile(
  optimizer = keras.optimizers.SGD(learning_rate=0.01),
  loss = 'binary_crossentropy', metrics = ['accuracy']
)

modelo.summary()

In [ ]:
history = modelo.fit(P, T, epochs=3, batch_size=16, verbose=True, validation_split=0.2)